In [1]:
# TensorFlow and tf.keras
import tensorflow as tf

import keras
import keras.backend
from keras import layers
from keras import models
from keras import optimizers
import keras.utils

print(tf.__version__)

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import sklearn
from sklearn.utils import shuffle

import os
import pickle

/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

1.13.1


In [2]:
#import network and feature list
import sys
sys.path.append('../python/')
from networkBuilder import *

## Import data:
Using Train and Test set produced by preprocessing notebook.

In [3]:
grid = 16
train_sig_features = np.load('/mnt/data/ml/PreProcessing/ShowJets_train_Zbb_exts_msd50to150_v2.npz')
train_bkg_features = np.load('/mnt/data/ml/PreProcessing/ShowJets_train_QCD_exts_msd50to150_v2.npz')
train_sig_images = np.load('/mnt/data/ml/PreProcessing/ShowJets_train_Zbb_exts_msd50to150_images_v2.npz')
train_bkg_images = np.load('/mnt/data/ml/PreProcessing/ShowJets_train_QCD_exts_msd50to150_images_v2.npz')

In [4]:
n_test = len(train_sig_features['jetPt'])
print(n_test)

1243305


In [5]:
train_sig = {**train_sig_images, **train_sig_features}
train_bkg = {**train_bkg_images, **train_bkg_features}

# Build CNN

## Build Model with only Images

In [ ]:
# features will be loaded into X
features = ['jetImages']

# label into Y
label = ['labels']
X_train, Y_train = build_XY(features, label, train_sig_images, train_bkg_images, "Zbb")

In [ ]:
CNN = build_CNN_2D(X_train, grid)

In [ ]:
X_batch = [ele[:10] for ele in X_train]
Y_batch = [ele[:10] for ele in Y_train]
example_result = CNN.predict(x = X_batch)
results = CNN.evaluate(x = X_batch, y = Y_batch )
print(example_result)
print(results)

In [ ]:
checkpoint_path = "model/CNN_Images_defaultopt.h1"
if not os.path.exists("model"):
    os.mkdir("model")

model_checkpoint = keras.callbacks.ModelCheckpoint(checkpoint_path, monitor = 'val_loss', 
                                                   verbose = 1, save_best_only = True, 
                                                   save_weights_only = False, mode = 'auto', 
                                                   period = 1)    
EPOCHS = 80
early_stop = keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 10)
history = CNN.fit(
    X_train, Y_train,
    epochs = EPOCHS, 
    validation_split = 0.2,
    verbose = 1,
    callbacks = [early_stop, model_checkpoint])

In [ ]:
if not os.path.exists("history"):
    os.mkdir("history")
with open('history/CNN_Images_defaultopt.history', 'wb') as file_pi:
    pickle.dump(history.history, file_pi)

## Build Model with Images + 5 XAUGS

### Select Features for Model

Show all possible features

In [ ]:
features = ['jetImages', 'dz_max', 'dxy_max','jetMassSD', 'tau2_sd_b05', 'tau3_b10']

In [ ]:
label = ['labels']

In [ ]:
features

In [ ]:
X_train, Y_train = build_XY(features, label, train_sig, train_bkg, "Zbb")

Initialize model:

In [ ]:
CNN = build_CNN_2D(X_train, grid)

Test model's prediction $before$ training:

In [ ]:
X_batch = [ele[:10] for ele in X_train]
Y_batch = [ele[:10] for ele in Y_train]
example_result = CNN.predict(x = X_batch)
results = CNN.evaluate(x = X_batch, y = Y_batch )
print(example_result)
print(results)

Now train! (warning: if building CNN, computer tends to get loud)

In [ ]:
checkpoint_path = "model/CNN_5XAUG.h1"
if not os.path.exists("model"):
    os.mkdir("model")

model_checkpoint = keras.callbacks.ModelCheckpoint(checkpoint_path, monitor = 'val_loss', 
                                                   verbose = 1, save_best_only = True, 
                                                   save_weights_only = False, mode = 'auto', 
                                                   period = 1)    
EPOCHS = 80
early_stop = keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 10)
history = CNN.fit(
    X_train, Y_train,
    epochs = EPOCHS,
    validation_split = 0.2,
    verbose = 1,
    callbacks = [early_stop, model_checkpoint])

In [ ]:
with open('history/CNN_5XAUG.h1.history', 'wb') as file_pi:
    pickle.dump(history.history, file_pi)

## Build Model with all Features

### Select Features for Model

Show all possible features

In [ ]:
# features will be loaded into X
features = get_feat(True, True, True, False) #xaugs, flavor, images, particleInfo

# label into Y
label = ['labels']

In [ ]:
X_train, Y_train = build_XY(features, label, train_sig, train_bkg, "Zbb")

Initialize model:

In [ ]:
CNN = build_CNN_2D(X_train, grid)

Test model's prediction $before$ training:

In [ ]:
X_batch = [ele[:10] for ele in X_train]
Y_batch = [ele[:10] for ele in Y_train]
example_result = CNN.predict(x = X_batch)
results = CNN.evaluate(x = X_batch, y = Y_batch )
print(example_result)
print(results)

Now train! (warning: if building CNN, computer tends to get loud)

In [ ]:
checkpoint_path = "model/CNN_all.h1"
if not os.path.exists("model"):
    os.mkdir("model")

model_checkpoint = keras.callbacks.ModelCheckpoint(checkpoint_path, monitor = 'val_loss', 
                                                   verbose = 1, save_best_only = True, 
                                                   save_weights_only = False, mode = 'auto', 
                                                   period = 1)    
EPOCHS = 80
early_stop = keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 10)
history = CNN.fit(
    X_train, Y_train,
    epochs = EPOCHS,
    validation_split = 0.2,
    verbose = 1,
    callbacks = [early_stop, model_checkpoint])

In [ ]:
with open('history/CNN_all.h1.history', 'wb') as file_pi:
    pickle.dump(history.history, file_pi)

In [ ]:
X_train, Y_train = build_XY(features, label, train_sig, train_bkg, "Zbb")

In [ ]:
CNN = build_CNN_2D(X_train, grid)

In [ ]:
X_batch = [ele[:10] for ele in X_train]
Y_batch = [ele[:10] for ele in Y_train]
example_result = CNN.predict(x = X_batch)
results = CNN.evaluate(x = X_batch, y = Y_batch )
print(example_result)
print(results)

In [ ]:
checkpoint_path = "model/CNN_all.h3"
if not os.path.exists("model"):
    os.mkdir("model")

model_checkpoint = keras.callbacks.ModelCheckpoint(checkpoint_path, monitor = 'val_loss', 
                                                   verbose = 1, save_best_only = True, 
                                                   save_weights_only = False, mode = 'auto', 
                                                   period = 1)    
EPOCHS = 80
early_stop = keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 10)
history = CNN.fit(
    X_train, Y_train,
    epochs = EPOCHS,
    validation_split = 0.2,
    verbose = 1,
    callbacks = [early_stop, model_checkpoint])

In [ ]:
with open('history/CNN_all.h3.history', 'wb') as file_pi:
    pickle.dump(history.history, file_pi)

In [ ]:
X_train, Y_train = build_XY(features, label, train_sig, train_bkg, "Zbb")

In [ ]:
CNN = build_CNN_2D(X_train, grid)

In [ ]:
X_batch = [ele[:10] for ele in X_train]
Y_batch = [ele[:10] for ele in Y_train]
example_result = CNN.predict(x = X_batch)
results = CNN.evaluate(x = X_batch, y = Y_batch )
print(example_result)
print(results)

In [ ]:
checkpoint_path = "model/CNN_all.h2"
if not os.path.exists("model"):
    os.mkdir("model")

model_checkpoint = keras.callbacks.ModelCheckpoint(checkpoint_path, monitor = 'val_loss', 
                                                   verbose = 1, save_best_only = True, 
                                                   save_weights_only = False, mode = 'auto', 
                                                   period = 1)    
EPOCHS = 80
early_stop = keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 10)
history = CNN.fit(
    X_train, Y_train,
    epochs = EPOCHS,
    validation_split = 0.2,
    verbose = 1,
    callbacks = [early_stop, model_checkpoint])

In [ ]:
with open('history/CNN_all.h2.history', 'wb') as file_pi:
    pickle.dump(history.history, file_pi)

In [ ]:
X_train, Y_train = build_XY(features, label, train_sig, train_bkg, "Zbb")

In [ ]:
CNN = build_CNN_2D(X_train, grid)

In [ ]:
X_batch = [ele[:10] for ele in X_train]
Y_batch = [ele[:10] for ele in Y_train]
example_result = CNN.predict(x = X_batch)
results = CNN.evaluate(x = X_batch, y = Y_batch )
print(example_result)
print(results)

In [ ]:
checkpoint_path = "model/CNN_all.h4"
if not os.path.exists("model"):
    os.mkdir("model")

model_checkpoint = keras.callbacks.ModelCheckpoint(checkpoint_path, monitor = 'val_loss', 
                                                   verbose = 1, save_best_only = True, 
                                                   save_weights_only = False, mode = 'auto', 
                                                   period = 1)    
EPOCHS = 80
early_stop = keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 10)
history = CNN.fit(
    X_train, Y_train,
    epochs = EPOCHS,
    validation_split = 0.2,
    verbose = 1,
    callbacks = [early_stop, model_checkpoint])

In [ ]:
with open('history/CNN_all.h4.history', 'wb') as file_pi:
    pickle.dump(history.history, file_pi)

### Select Features for Model

Show all possible features

In [ ]:
features = get_feat(True, False, False, False) #xaugs, flavor, images, particleInfo
print(features)

In [ ]:
X_train, Y_train = build_XY(features, label, train_sig, train_bkg,"Zbb")

Initialize model:

In [ ]:
CNN = build_DNN()

Test model's prediction $before$ training:

In [ ]:
X_batch = [ele[:10] for ele in X_train]
Y_batch = [ele[:10] for ele in Y_train]
example_result = CNN.predict(x = X_batch)
results = CNN.evaluate(x = X_batch, y = Y_batch )
print(example_result)
print(results)

Now train! (warning: if building CNN, computer tends to get loud)

In [ ]:
checkpoint_path = "model/DNN_noflavour.h1"
if not os.path.exists("model"):
    os.mkdir("model")

model_checkpoint = keras.callbacks.ModelCheckpoint(checkpoint_path, monitor = 'val_loss', 
                                                   verbose = 1, save_best_only = True, 
                                                   save_weights_only = False, mode = 'auto', 
                                                   period = 1)    
EPOCHS = 60
early_stop = keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 10, min_delta = 1e-5)
history = CNN.fit(
    X_train, Y_train,
    epochs = EPOCHS,
    validation_split = 0.2,
    verbose = 1,
    callbacks = [early_stop, model_checkpoint])

In [ ]:
with open('history/DNN_noflavour.h1.history', 'wb') as file_pi:
    pickle.dump(history.history, file_pi)

In [ ]:
label = ['labels']

In [ ]:
features = get_feat(True, True, False, False) #xaugs, flavor, images, particleInfo

In [ ]:
print(features)

In [ ]:
X_train, Y_train = build_XY(features, label, train_sig_features, train_bkg_features,"Zbb")

In [ ]:
CNN = build_DNN(X_train)

In [ ]:
X_batch = [ele[:10] for ele in X_train]
Y_batch = [ele[:10] for ele in Y_train]
example_result = CNN.predict(x = X_batch)
results = CNN.evaluate(x = X_batch, y = Y_batch )
print(example_result)
print(results)

In [ ]:
Y_batch

In [ ]:
checkpoint_path = "model/DNN_all_defaultopt.h1"
if not os.path.exists("model"):
    os.mkdir("model")

In [ ]:
run_DNN(CNN, X_train, Y_train, checkpoint_path)

In [ ]:
with open('history/DNN_all_defaultopt.h1.history', 'wb') as file_pi:
    pickle.dump(history.history, file_pi)